In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
LOG_PATH="/Users/law/repos/ma/disco/benchmark-runs/runs_sustainable_throughput"

In [12]:
import os
import re

RUN_RE = re.compile(r"Running (\d+) child.*, (\d+) stream.*")
THROUGHPUT_RE = re.compile(r"Found sustainable candidate \((\d+) events/s.\)*")

def parse_log_file(log_file):
    sustainable_throughputs= {}
    
    current_run = None
    current_throughput = None
    with open(log_file) as f:
        for line in f:
            run_match = RUN_RE.match(line)
            if run_match is not None:
                if current_run != None:
                    print(f"Did not find candidate line for {current_run}")
                current_run = (int(run_match.group(1)), int(run_match.group(2)))
                current_throughput = None

            throughput_match = THROUGHPUT_RE.match(line)
            if throughput_match is not None:
                if current_throughput is not None:
                    print(f"Did not find run line after {current_run}")
                current_throughput = int(throughput_match.group(1))
                sustainable_throughputs[current_run] = current_throughput
                current_run = None
                
    if current_run is not None:
        print(f"Did not find candidate line for {current_run}")
                
    return sustainable_throughputs
                

ALL_THROUGHPUTS = {}
for log_file in sorted(os.listdir(LOG_PATH)):
    if log_file.endswith("-nodes-max.log"):
        print(f"Parsing {log_file}")
        sustainable_throughputs = parse_log_file(os.path.join(LOG_PATH, log_file))
        print(f"current: {sustainable_throughputs}")
        ALL_THROUGHPUTS = {**ALL_THROUGHPUTS, **sustainable_throughputs}
        print(f"all:     {ALL_THROUGHPUTS}\n")        

Parsing sustainable_run_2019-08-28-1044-1-child-2to8-nodes-max.log
current: {(1, 2): 492187, (1, 4): 257812, (1, 8): 117187}
all:     {(1, 2): 492187, (1, 4): 257812, (1, 8): 117187}

Parsing sustainable_run_2019-08-28-1140-2to4-children-16-nodes-max.log
current: {(2, 2): 578125, (2, 4): 296875, (4, 4): 554687, (4, 8): 281250, (8, 8): 226562, (8, 16): 101562}
all:     {(1, 2): 492187, (1, 4): 257812, (1, 8): 117187, (2, 2): 578125, (2, 4): 296875, (4, 4): 554687, (4, 8): 281250, (8, 8): 226562, (8, 16): 101562}

Parsing sustainable_run_2019-08-28-1703-2to4-children-2to8-nodes-max.log
Did not find candidate line for (8, 16)
current: {(2, 2): 1085937, (2, 4): 742187, (4, 4): 1031250, (4, 8): 718750, (8, 8): 945312}
all:     {(1, 2): 492187, (1, 4): 257812, (1, 8): 117187, (2, 2): 1085937, (2, 4): 742187, (4, 4): 1031250, (4, 8): 718750, (8, 8): 945312, (8, 16): 101562}

Parsing sustainable_run_2019-08-28-1817-8-children-16-nodes-max.log
current: {(8, 16): 687500}
all:     {(1, 2): 492187

In [13]:
for (num_children, num_streams), throughput in sorted(ALL_THROUGHPUTS.items()):
    print(f"Total sustainable throughput for {num_children} child(ren) with " \
          f"{num_streams // num_children} stream(s) each " \
          f"is {throughput * num_streams} events/s.")

Total sustainable throughput for 1 child(ren) with 1 stream(s) each is 1312500 events/s.
Total sustainable throughput for 1 child(ren) with 2 stream(s) each is 1421874 events/s.
Total sustainable throughput for 1 child(ren) with 4 stream(s) each is 1562500 events/s.
Total sustainable throughput for 1 child(ren) with 8 stream(s) each is 1500000 events/s.
Total sustainable throughput for 2 child(ren) with 1 stream(s) each is 2203124 events/s.
Total sustainable throughput for 2 child(ren) with 2 stream(s) each is 2968748 events/s.
Total sustainable throughput for 4 child(ren) with 1 stream(s) each is 3468748 events/s.
Total sustainable throughput for 4 child(ren) with 2 stream(s) each is 5750000 events/s.
Total sustainable throughput for 8 child(ren) with 1 stream(s) each is 5937496 events/s.
Total sustainable throughput for 8 child(ren) with 2 stream(s) each is 11000000 events/s.


# Plots

In [ ]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True, 'pgf.rcfonts' : False})